In [5]:
# Change to Project Folder
cd '/content/drive/Othercomputers/My MacBook Pro/Academic Projects/DB Project'

SyntaxError: ignored

In [ ]:
# install transfomer library
!pip install transformers

In [ ]:
# Import Statements
import pandas as pd
import re
import pandas as pd
import re
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [ ]:
# Reading Sentiment Data

df_twitter1 = pd.read_csv('2020-07-09 till 2020-09-19.csv')
df_twitter2 = pd.read_csv('2020-09-20 till 2020-10-13.csv')

# Concatenating them
df = pd.concat([df_twitter1, df_twitter2])

df

In [ ]:
# Reading Match Result Data

df_match_result = pd.read_csv('premier-league-matches.csv')

# Convert the 'Date' column to datetime
df_match_result['Date'] = pd.to_datetime(df_match_result['Date'])

# Define  date range based on available sentiment data
start_date = '2020-07-09'
end_date = '2020-10-13'

# Filter the data
df_match_result_sliced = df_match_result[(df_match_result['Date'] >= start_date) & (df_match_result['Date'] <= end_date)]


df_match_result_sliced

In [ ]:
# Modifying Sentiment Data - Getting in sentiment data spanning over previous 7 days for every single match date in Match Result data

# Convert your DataFrame date column to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

#  list of dates
date_list = df_match_result_sliced.Date.unique()
date_list = pd.to_datetime(date_list)  # convert list to datetime

# Function to check if a date is within 7 days of any date in the list
def is_within_7_days(row_date):
    return any(abs(row_date - list_date) <= pd.Timedelta(days=7) for list_date in date_list)

# Apply the function to filter the DataFrame
df_sentiment = df[df['created_at'].apply(is_within_7_days)]

df_sentiment

In [ ]:
# Clean the tweets
def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet


df_sentiment['clean_text'] = df_sentiment['text'].apply(clean_tweet)

In [ ]:
# Model Loading

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = TFAutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

In [ ]:
# Prediction Code - Generating Sentiments (Score as well as label) using Model

def batch_predict(texts, batch_size=1000):  # Increased batch size
    predictions = []
    for i in tqdm(range(0, len(texts), batch_size)):  # tqdm for progress tracking
        batch = texts[i:i+batch_size]
        encoded_batch = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="tf")
        outputs = model(encoded_batch['input_ids'], attention_mask=encoded_batch['attention_mask'])
        batch_predictions = tf.nn.softmax(outputs.logits, axis=-1)
        predictions.extend(batch_predictions.numpy().tolist())
    return predictions

# Predict sentiments in batches
sentiment_scores = batch_predict(df_sentiment['clean_text'].tolist(), batch_size=1000)  # Use .tolist() directly

# Adding score to df
df_sentiment['sentiment_scores'] = sentiment_scores

# Extract the sentiment labels from the sentiment scores
df_sentiment['sentiment'] = [np.argmax(score) for score in sentiment_scores]
df_sentiment['sentiment label'] = df_sentiment['sentiment'].map({0: 'Negative', 1: 'Neutral', 2: 'Positive'})


df_sentiment

In [ ]:
# Saving Sentiments Data
df_sentiment['created_at'] = pd.to_datetime(df_sentiment['created_at'], errors='coerce')
df_sentiment.to_csv('twitter_sentiment_analysis.csv', index=False)

# Saving Match Result Data
df_match_result_sliced.to_csv('match_result.csv', index=False)

In [ ]:
# Reading Bets Data

df_bet1 = pd.read_csv('E0.csv')

df_bet2 = pd.read_csv('2019-2020.csv')

# Concatenating them
df_bet = pd.concat([df_bet1, df_bet2])

df_bet

In [ ]:
# Extracting Relevant Bets Data

# Convert the 'Date' column to datetime
df_bet['Date'] = pd.to_datetime(df_bet['Date'])

# Define date range
start_date = '2020-07-09'
end_date = '2020-10-13'

# Filter the data
df_bets_data = df_bet[(df_bet['Date'] >= start_date) & (df_bet['Date'] <= end_date)]

df_bets_data.to_csv('Bets_Data.csv', index=False)

In [ ]:
# Read all data
df_sentiment = pd.read_csv('twitter_sentiment_analysis.csv')
df_bets_data = pd.read_csv('Bets_Data.csv')
df_match_result = pd.read_csv('match_result.csv')

In [ ]:
# Team Mapping to maintain uniformity accross Datasets
team_name_mapping = {
    'West Brom': 'West Bromwich Albion',
    'Tottenham': 'Tottenham Hotspur',
    'Brighton': 'Brighton & Hove Albion',
    'Sheffield United': 'Sheffield United',
    'Sheffield Utd': 'Sheffield United',
    'Everton': 'Everton',
    'Leeds': 'Leeds United',
    'Leeds United': 'Leeds United',
    'Man United': 'Manchester United',
    'Manchester Utd': 'Manchester United',
    'Arsenal': 'Arsenal',
    'Southampton': 'Southampton',
    'Newcastle': 'Newcastle United',
    'Newcastle Utd': 'Newcastle United',
    'Chelsea': 'Chelsea',
    'Leicester': 'Leicester City',
    'Leicester City': 'Leicester City',
    'Aston Villa': 'Aston Villa',
    'Wolves': 'Wolverhampton Wanderers',
    'Crystal Palace': 'Crystal Palace',
    'Burnley': 'Burnley',
    'Man City': 'Manchester City',
    'Manchester City': 'Manchester City',
    'West Ham': 'West Ham United',
    'Fulham': 'Fulham',
    'Liverpool': 'Liverpool',
    'Bournemouth': 'AFC Bournemouth',
    'Norwich': 'Norwich City',
    'Norwich City': 'Norwich City',
    'Watford': 'Watford'
}


df_bets_data['HomeTeam'] = df_bets_data['HomeTeam'].map(team_name_mapping).fillna(df_bets_data['HomeTeam'])
df_bets_data['AwayTeam'] = df_bets_data['AwayTeam'].map(team_name_mapping).fillna(df_bets_data['AwayTeam'])


df_match_result['Home'] = df_match_result['Home'].map(team_name_mapping).fillna(df_match_result['Home'])
df_match_result['Away'] = df_match_result['Away'].map(team_name_mapping).fillna(df_match_result['Away'])


In [ ]:
# Merging Match Result and Bets Data

# Convert dates to datetime objects for accurate merging
df_match_result['Date'] = pd.to_datetime(df_match_result['Date'])
df_bets_data['Date'] = pd.to_datetime(df_bets_data['Date'])

# Merge df_match_result with df_bets_data
merged_df = pd.merge(df_match_result, df_bets_data,
                     left_on=['Date', 'Home', 'Away'],
                     right_on=['Date', 'HomeTeam', 'AwayTeam'])
merged_df

In [ ]:
# Merging merge_df with the sentiment data - Averaging out across the  relevant sentiment data(7 days)

# Dropping redundant columns
merged_df.drop(columns=['HomeTeam', 'AwayTeam'], inplace=True)

# Rename betting odds columns for clarity
merged_df.rename(columns={'AvgH': 'AvgOdds_HomeWin', 'AvgD': 'AvgOdds_Draw', 'AvgA': 'AvgOdds_AwayWin'}, inplace=True)


# Cell execution start
def calculate_sentiment_scores(df, team_name, start_date):
    end_date = start_date - pd.Timedelta(days=7)
    team_sentiments = df[(df['file_name'] == team_name) &
                         (df['created_at'] >= end_date) &
                         (df['created_at'] < start_date)]

    # Initialize sums for each sentiment
    positive_sum = neutral_sum = negative_sum = 0

    # Check if there are sentiment data points available
    if not team_sentiments.empty:
        # Debugging: Print first few sentiment scores
        print(team_sentiments['sentiment_scores'].head())

        # Assuming sentiment_scores is a string representation of a list
        for sentiment_score_str in team_sentiments['sentiment_scores']:
            try:
                sentiment_scores = eval(sentiment_score_str)  # Convert string to list
                positive_sum += sentiment_scores[2]
                neutral_sum += sentiment_scores[1]
                negative_sum += sentiment_scores[0]
            except Exception as e:
                print(f"Error processing sentiment score {sentiment_score_str}: {e}")

    return positive_sum, neutral_sum, negative_sum


# Cell execution start
for index, row in merged_df.iterrows():
    match_date = row['Date']
    home_team = row['Home']
    away_team = row['Away']

    home_positive, home_neutral, home_negative = calculate_sentiment_scores(df_sentiment, home_team, match_date)
    away_positive, away_neutral, away_negative = calculate_sentiment_scores(df_sentiment, away_team, match_date)

    merged_df.at[index, 'HomeTeam_PositiveSentiment'] = home_positive
    merged_df.at[index, 'HomeTeam_NeutralSentiment'] = home_neutral
    merged_df.at[index, 'HomeTeam_NegativeSentiment'] = home_negative
    merged_df.at[index, 'AwayTeam_PositiveSentiment'] = away_positive
    merged_df.at[index, 'AwayTeam_NeutralSentiment'] = away_neutral
    merged_df.at[index, 'AwayTeam_NegativeSentiment'] = away_negative


# Cell execution start
print_columns = ['Date', 'Home', 'Away',
                 'HomeTeam_PositiveSentiment', 'HomeTeam_NeutralSentiment', 'HomeTeam_NegativeSentiment',
                 'AwayTeam_PositiveSentiment', 'AwayTeam_NeutralSentiment', 'AwayTeam_NegativeSentiment',
                 'AvgOdds_HomeWin', 'AvgOdds_Draw', 'AvgOdds_AwayWin',
                 'HomeGoals', 'AwayGoals']

# Displaying the merged dataframe with specified columns
merged_df = merged_df[print_columns]






In [ ]:
# Scalling the dataset

from sklearn.preprocessing import MinMaxScaler

# Columns to be scaled
sentiment_cols = ['HomeTeam_PositiveSentiment', 'HomeTeam_NeutralSentiment', 'HomeTeam_NegativeSentiment',
                  'AwayTeam_PositiveSentiment', 'AwayTeam_NeutralSentiment', 'AwayTeam_NegativeSentiment']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the sentiment columns
merged_df[sentiment_cols] = scaler.fit_transform(merged_df[sentiment_cols])


# Cell execution start
for col in sentiment_cols:
    # Calculate the mean of the column excluding zeros
    mean_value = merged_df[merged_df[col] != 0][col].mean()

    # Replace zeros with the mean value
    merged_df[col] = merged_df[col].replace(0, mean_value)


merged_df




In [ ]:
# Saving the training data
merged_df.to_csv('train_data.csv')


In [4]:
!zip -r Data Data.zip

	zip warning: name not matched: Data.zip

zip error: Nothing to do! (try: zip -r Data . -i Data.zip)
